<a href="https://colab.research.google.com/github/KimEunChae9506/2024-smartfarm-AI-strongberry/blob/main/111.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.metrics import r2_score
import chardet

# 데이터 로드
file_path = '/content/dataset.csv'
data = pd.read_csv(file_path)


#f = open(file_path,'rt',encoding='UTF-8')
#reader = csv.reader(f)

#once contents are available, I then put them in a list
#csv_list = []
#for l in reader:
#    csv_list.append(l)
#f.close()
#now pandas has no problem getting into a df
#data = pd.DataFrame(csv_list)

# 데이터 확인
#print(data.head())
#print(data.info())
#print(data.describe())

# 데이터 전처리 (결측값 처리, 필요시 인코딩 등)
data = data.dropna()

# 설명 변수(X)와 목표 변수(y) 분리
# 'target'이라는 열이 목표 변수라고 가정합니다. 필요에 따라 수정하세요.
X = data.drop('outtrn_cumsum', axis=1)
y = data['outtrn_cumsum']

# 데이터 분할 (학습용 데이터와 테스트용 데이터)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 데이터 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 모델 리스트
models = {
    "Random Forest": RandomForestRegressor(random_state=42)
}

# 모델 성능 평가
results = {}
for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    y_train_pred = model.predict(X_train_scaled)
    y_test_pred = model.predict(X_test_scaled)

    train_r2 = r2_score(y_train, y_train_pred)
    test_r2 = r2_score(y_test, y_test_pred)

    results[name] = {
        "Train R2": train_r2,
        "Test R2": test_r2
    }

# 결과 출력
for model_name, result in results.items():
    print(f"{model_name}:")
    print(f"  Training R2: {result['Train R2']:.4f}")
    print(f"  Test R2: {result['Test R2']:.4f}\n")


Random Forest:
  Training R2: 0.9984
  Test R2: 0.9912



In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# RMSE 계산 함수 정의
def calculate_rmse(targets, predictions):
    return np.sqrt(mean_squared_error(targets, predictions))

# R2 스코어 계산 함수 정의
def calculate_R2_score(y_test, y_pred):
    return r2_score(y_test, y_pred)

# 데이터 불러오기
input_data = pd.read_csv('/content/dataset2.csv')

# 데이터 전처리
# 'frmDist' 열을 기준으로 데이터 그룹화 및 평균값 계산하여 'group_df'에 저장
group_df = input_data.groupby('frmDist').mean()

print(group_df)
print('=======')
# 'frmDist' 열을 기준으로 데이터 다시 그룹화하고, 'outtrn_cumsum' 열에서 최댓값 계산하여 'group_outtrn_cumsum_df'에 저장
group_outtrn_cumsum_df = input_data.groupby('frmDist').max()[['outtrn_cumsum']]
print(group_outtrn_cumsum_df)
print('=======')

# 'group_df'에서 'outtrn_cumsum'과 'date' 열 제거
group_df.drop(['outtrn_cumsum', 'date'], axis=1, inplace=True)

# 'group_df'와 'group_outtrn_cumsum_df'를 합쳐 'merge_df' 데이터프레임 생성
merge_df = pd.concat([group_df, group_outtrn_cumsum_df], axis=1)
print(merge_df)
print('=======')

# 'merge_df'에서 입력 피처(독립 변수)와 타겟 변수(종속 변수) 분리
X = merge_df.drop(['outtrn_cumsum', 'HeatingEnergyUsage_cumsum'], axis=1)
Y = merge_df.loc[:,['outtrn_cumsum', 'HeatingEnergyUsage_cumsum']]

# 데이터 분할: 훈련 데이터와 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=42, test_size=0.2)

# 데이터 표준화
ss = StandardScaler()
ss.fit(X_train)

train_scaled = ss.transform(X_train)
test_scaled = ss.transform(X_test)

# ExtraTreesRegressor 모델 설정 및 하이퍼파라미터 조정
model = ExtraTreesRegressor(
    n_estimators=400,
    max_depth=5,  # 최대 트리 깊이를 제한
    min_samples_split=4,  # 노드를 분할하기 위한 최소 샘플 수
    min_samples_leaf=1,  # 리프 노드에 필요한 최소 샘플 수
    random_state=42
)

# 모델 훈련
model.fit(train_scaled, y_train)

# 모델을 사용하여 테스트 데이터에 대한 예측 수행
y_pred = model.predict(test_scaled)

# RMSE 및 R2 스코어 계산
rmse = calculate_rmse(y_test, y_pred)
r2score = calculate_R2_score(y_test, y_pred)

# 결과 출력
print("RMSE:", rmse)
print("R2_score:", r2score)


                  date       inTp       inHd  otmsuplyqy     acSlrdQy  \
frmDist                                                                 
DBSF1059  2.022108e+07   0.000000   0.000000    0.000000     0.000000   
DBSF1093  2.022108e+07   0.000000   0.000000    0.000000     0.000000   
DBSF1121  2.022108e+07   0.000000   0.000000    0.000000     0.000000   
DBSF1172  2.022108e+07   0.000000   0.000000    0.000000     0.000000   
DBSF1180  2.022108e+07   0.000000   0.000000    0.000000     0.000000   
...                ...        ...        ...         ...          ...   
DBSF9922  2.018619e+07  18.556063  78.618986  102.310054  1214.383442   
DBSF9942  2.017694e+07  17.730452  79.149949   58.567463  1085.351211   
DBSF9948  2.022108e+07   0.000000   0.000000    0.000000     0.000000   
DBSF9961  2.017593e+07  19.905136  82.866854   19.867339  1352.348912   
DBSF9969  2.017627e+07  19.358895  80.710109  107.747260  1369.541307   

               cunt        ph      outTp     outWs

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# RMSE 계산 함수 정의
def calculate_rmse(targets, predictions):
    return np.sqrt(mean_squared_error(targets, predictions))

# R2 스코어 계산 함수 정의
def calculate_R2_score(y_test, y_pred):
    return r2_score(y_test, y_pred)

# 데이터 불러오기
input_data = pd.read_csv('/content/dataset2.csv')

# 데이터 전처리
# 'frmDist' 열을 기준으로 데이터 그룹화 및 평균값 계산하여 'group_df'에 저장
group_df = input_data.groupby('frmDist').mean()

# 'frmDist' 열을 기준으로 데이터 다시 그룹화하고, 'outtrn_cumsum' 열에서 최댓값 계산하여 'group_outtrn_cumsum_df'에 저장
group_outtrn_cumsum_df = input_data.groupby('frmDist').max()[['outtrn_cumsum']]

# 'group_df'에서 'outtrn_cumsum'과 'date' 열 제거
group_df.drop(['outtrn_cumsum', 'date'], axis=1, inplace=True)

# 'group_df'와 'group_outtrn_cumsum_df'를 합쳐 'merge_df' 데이터프레임 생성
merge_df = pd.concat([group_df, group_outtrn_cumsum_df], axis=1)

# 'merge_df'에서 입력 피처(독립 변수)와 타겟 변수(종속 변수) 분리
X = merge_df.drop(['outtrn_cumsum', 'HeatingEnergyUsage_cumsum'], axis=1)
Y = merge_df.loc[:,['outtrn_cumsum', 'HeatingEnergyUsage_cumsum']]

# 데이터 분할: 훈련 데이터와 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=42, test_size=0.2)

# 데이터 표준화
ss = StandardScaler()
ss.fit(X_train)

train_scaled = ss.transform(X_train)
test_scaled = ss.transform(X_test)

# ExtraTreesRegressor 모델 설정 및 하이퍼파라미터 조정
model = (RandomForestRegressor(random_state=42))

# 모델 훈련
model.fit(train_scaled, y_train)

# 모델을 사용하여 테스트 데이터에 대한 예측 수행
y_pred = model.predict(test_scaled)

# RMSE 및 R2 스코어 계산
rmse = calculate_rmse(y_test, y_pred)
r2score = calculate_R2_score(y_test, y_pred)

# 결과 출력
print("RMSE:", rmse)
print("R2_score:", r2score)


RMSE: 16905.883612315705
R2_score: 0.9988630429769426


In [ ]:
pwd

'/content'